In [2]:
import os
import warnings
warnings.filterwarnings("ignore")

from typing import List, Literal
from typing_extensions import TypedDict
from dotenv import load_dotenv

# LangChain 관련 임포트
from langchain_core.documents import Document
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

# LangGraph 관련 임포트
from langgraph.graph import StateGraph, START, END

# 환경설정
load_dotenv()

if not os.environ.get('OPENAI_API_KEY'):
    raise ValueError('key check....')


In [14]:

# 조건부 엣지가 포함된 그래프
def conditional_graph():    
    '''조건부 엣지가 포함된 LangGraph
    검색결과에 따라 다른 경로로 분기'''
    # 상태 정의
    class ConditionalState(TypedDict):
        question:str
        documents : List[Document]
        search_type : str
        answer : str
    # 내부문서  LoadeText or .....
    INTERNAL_DOCS = {
            "회사" : [Document(page_content="우리 회사의 AI전략은 RAG 시스템 구축입니다.")],
            "정책" : [Document(page_content="사내 데이터 보안 정책은 외부 공유 금지입니다.")],
    }
    # 노드함수들을 구현
    def internal_search_node(state:ConditionalState) -> dict:
        '''내부 문서 검색'''
        question = state['question']
        documents = []
        for keyword,docs in INTERNAL_DOCS.items():
            if keyword in question:
                documents.extend(docs)
        return {'documents':documents,'search_type':'internal'}
    
    def web_search_node(state:ConditionalState) -> dict:
        '''웹 검색(시뮬레이션)'''
        mock_result = Document(
        page_content= f"{state['question']}에 대한 웹 검색 결과 입니다.",
        metadata = {'source':'web'}
        )
        return {'documents':[mock_result],'search_type':'web'}
    def generate_node(state:ConditionalState) -> dict:  # LLM 을 이용해 문서 내용을 기반으로 답변 생성
        '''답변 생성'''
        llm = ChatOpenAI(model='gpt-4o-mini',temperature=0)
        context = '\n'.join([ doc.page_content for doc in state['documents']  ])
        prompt = ChatPromptTemplate.from_template(
            '''컨텍스트:{context}\n\n 질문:{question}\n\n답변:'''
        )
        chain = prompt | llm | StrOutputParser()
        answer = chain.invoke({'context':context, 'question': state['question']})
        return {'answer' : f"[{state['search_type']}] {answer}"}
    
    # 조건 함수
    def decide_search_type(state:ConditionalState) -> Literal['generate','web_search']:
        '''검색결과에 따라 분기'''
        if state['documents']:
            return 'generate'
        else:
            return 'web_search'
    
    # 그래프 구축
    graph = StateGraph(ConditionalState)
    graph.add_node('internal_search', internal_search_node)
    graph.add_node('web_search', web_search_node)
    graph.add_node('generate', generate_node)

    graph.add_edge(START,'internal_search')
    graph.add_conditional_edges(
        'internal_search',
        decide_search_type,
        {
            'generate' :'generate',
            'web_search':'web_search'
        }
    )
    graph.add_edge('web_search','generate')
    graph.add_edge('generate', END)

    app = graph.compile()

    # 테스트 1 : 내부문서에 있는 질문
    print('\n[테스트1] 내부 문서가 존재하는 경우')
    result1 = app.invoke({
        'question' : '회사 AI 전략은?',
        'documents':[],
        'search_type':'',
        'answer':''
    })
    print(f"답변 : {result1['answer']}")
    # 테스트 2 : 내부문서에 없는 질문
    print('\n[테스트2] 내부 문서가 없는 경우 -> 웹 검색')
    result2 = app.invoke({
        'question' : '오늘날씨는??',
        'documents':[],
        'search_type':'',
        'answer':''
    })
    print(f"답변 : {result2['answer']}")

# 조건부 분기 테스트
conditional_graph()



[테스트1] 내부 문서가 존재하는 경우
답변 : [internal] 회사의 AI 전략은 RAG 시스템 구축입니다. RAG 시스템은 정보 검색과 생성 모델을 결합하여 효율적으로 데이터를 활용하고, 보다 정확하고 유용한 결과를 제공하는 것을 목표로 하고 있습니다. 이를 통해 고객의 요구에 맞춘 맞춤형 솔루션을 제공하고, 비즈니스 프로세스를 최적화할 계획입니다.

[테스트2] 내부 문서가 없는 경우 -> 웹 검색
답변 : [web] 오늘 날씨는 지역에 따라 다를 수 있습니다. 정확한 날씨 정보를 원하신다면, 현재 위치를 알려주시면 더 구체적인 정보를 제공해드릴 수 있습니다. 또는 기상청 웹사이트나 날씨 앱을 통해 실시간 날씨를 확인하실 수 있습니다.
